In [1]:
import os
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\sara\\medical-chatbot'

In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# extract text from pdf file
def load_pdf(data):
    loader= DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls= PyPDFLoader # type: ignore
        )
        
    documents = loader.load()
    return documents    
    

In [7]:
extracted_docs = load_pdf('data')

In [9]:
len(extracted_docs)

637

In [11]:
# Display the first 5 documents in the list
extracted_docs[:5]

[Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\gale_encyclopedia_of_medicine.pdf', 'total_pages': 637, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\gale_encyclopedia_of_medicine.pdf', 'total_pages': 637, 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'creator': 'PyPDF', 'creationdate': '2004-12-18T17:00:02-05:00', 'moddate': '2004-12-18T16:15:31-06:00', 'source': 'data\\gale_encyclopedia_of_medicine.pdf', 'total_pages': 637, 'page': 2, 'page_label': '3'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S.

In [12]:
from typing import List
from langchain.schema import Document

# include only relevant content from the documents
def filter_docs(docs: List[Document]) -> List[Document]:
    filtered_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source") # Extract the 'source' from the original metadata
        filtered_docs.append(
            Document(
                page_content=doc.page_content,   # Keep the original page content
                metadata={"source": src}         # Only keep 'source' in metadata
            )
        )
    return filtered_docs    

In [13]:
filtered_docs = filter_docs(extracted_docs)

In [16]:
filtered_docs[20:25]

[Document(metadata={'source': 'data\\gale_encyclopedia_of_medicine.pdf'}, page_content='the late second or third trimester. Although D&X is high-\nly controversial, some physicians argue that it has advan-\ntages that make it a preferable procedure in some circum-\nstances. One perceived advantage is that the fetus is\nremoved largely intact, allowing for better evaluation and\nautopsy of the fetus in cases of known fetal anomalies.\nIntact removal of the fetus may also confer a lower risk\nof puncturing the uterus or damaging the cervix. Another\nperceived advantage is that D&X ends the pregnancy\nwithout requiring the woman to go through labor, which\nmay be less emotionally traumatic than other methods of\nlate-term abortion. In addition, D&X may offer a lower\ncost and shorter procedure time.\nPrecautions\nWomen considering D&X should be aware of the\nhighly controversial nature of this procedure. A contro-\nversy common to all late-term abortions is whether the\nfetus is viable, o

In [ ]:
#split documents into smaller chunks
def split_text(docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200
    )
    return text_splitter.split_documents(docs)


In [18]:
text_chunks= split_text(filtered_docs)

In [21]:
text_chunks[50:55]

[Document(metadata={'source': 'data\\gale_encyclopedia_of_medicine.pdf'}, page_content='Higher frequency ultrasound waves produce more\ndetailed images, but are also more readily absorbed and\nso cannot penetrate as deeply into the body. Abdominal\nultrasound imaging is generally performed at frequencies\nbetween 2-5 MHz.\nAn ultrasound machine consists of two parts: the\ntransducer and the analyzer. The transducer both produces\nthe sound waves that penetrate the body and receives the\nreflected echoes. Transducers are built around piezoelec-\ntric ceramic chips. (Piezoelectric refers to electricity that\nis produced when you put pressure on certain crystals\nsuch as quartz). These ceramic chips react to electric puls-\nes by producing sound waves ( they are transmitting\nwaves) and react to sound waves by producing electric\npulses (receiving). Bursts of high frequency electric puls-\nes supplied to the transducer causes it to produce the\nscanning sound waves. The transducer then re

In [22]:
len(text_chunks)

3426

In [23]:
from langchain.embeddings import HuggingFaceEmbeddings
import torch 
def download_embeddings(model_name: str) -> HuggingFaceEmbeddings:
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs={"device": "cuda" if torch.cuda.is_available() else "cpu"}
    )

In [ ]:
embeddings= download_embeddings("sentence-transformers/all-MiniLM-L6-v2")

C:\Users\sara\AppData\Local\Temp\ipykernel_9556\3456768351.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(
C:\Users\sara\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\sara\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Use

In [25]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={}, multi_process=False, show_progress=False)

In [26]:
from dotenv import load_dotenv
load_dotenv()

True

In [27]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [28]:
from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

In [29]:
pc

In [31]:
from pinecone import ServerlessSpec

index_name="medical-chatbot"
if not pc.has_index(index_name):
   pc.create_index(
      name= index_name,
      dimension= 384,
      metric= "cosine",
      spec= ServerlessSpec(cloud="aws", region="us-east-1")
   
   ) 
   
index = pc.Index(index_name)   


In [32]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

C:\Users\sara\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [33]:
# load existing index
docsearch = PineconeVectorStore.from_existing_index(
    index_name= index_name,
    embedding= embeddings
)

In [34]:
docsearch

In [39]:
retriever = docsearch.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 3} 
    )

In [41]:
retrieved_docs = retriever.invoke("What is the treatment for anemia?")
retrieved_docs

C:\Users\sara\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[Document(id='5745c43b-55fa-478a-aebe-8c57ffbca9bc', metadata={'source': 'data\\gale_encyclopedia_of_medicine.pdf'}, page_content='not require treatment. Those with severe thalassemia\nmay require bone marrow transplantation. Genetic thera-\npy is is being investigated and may soon be available.\nHemolytic anemia\nAcquired hemolytic anemia can generally be cured\nwhen the cause is removed.\nPrevention\nInherited anemias cannot be prevented. Genetic\ncounseling can help parents cope with questions and\nconcerns about transmitting disease-causing genes to\ntheir children.\nAvoiding excessive use of alcohol, eating a balanced\ndiet that contains plenty of iron-rich foods, and taking a\ndaily multivitamin can help prevent anemia.\nMethods of preventing specific types of anemia\ninclude:\n• avoiding lengthy exposure to industrial chemicals and\ndrugs known to cause aplastic anemia\n• not taking medication that has triggered hemolytic ane-\nmia and not eating foods that have caused hemolysis

In [42]:
from langchain_google_genai import GoogleGenerativeAI
model = GoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)


In [43]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [47]:
system_prompt = (
    "You are a medical assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [48]:
question_answer_chain= create_stuff_documents_chain(model, prompt)
rag_chain= create_retrieval_chain(retriever, question_answer_chain)

In [50]:
response = rag_chain.invoke({
    "input": "What is the treatment for anemia?"})
print(response['answer'])

C:\Users\sara\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


The treatment for anemia varies depending on the type and cause:

*   **Folic-acid and iron-deficiency anemias:** Usually treated with iron supplements, self-administered injections of vitamin B12, or oral folic acid replacements. Vitamin C deficiency anemia can be cured by taking one vitamin C tablet a day. Supplements may need to be continued for six months to replenish reserves.
*   **Anemia caused by excessive blood loss:** May require surgery to control bleeding, such as for heavy menstrual flow, a bleeding ulcer, or to remove polyps from the bowels. Transfusions of red blood cells may be used to accelerate red blood cell production.
*   **Thalassemia:** Patients usually do not require treatment. However, those with a severe form may need periodic hospitalization for blood transfusions and/or bone marrow transplantation. Genetic therapy is also being investigated.
*   **Sickle cell anemia:** Cannot be cured, but treatment involves regular eye examinations, immunizations for pneumo